In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用降低服务器进行分布式培训

## 概述

本笔记本演示了如何使用 Vertex Training Reduction Server 优化大规模分布式训练工作。使用 TensorFlow Model Garden 中的 [TensorFlow NLP Modelling Toolkit](https://github.com/tensorflow/models/tree/master/official/nlp#tensorflow-nlp-modelling-toolkit) 预处理数据并创建模型和训练循环。使用 `tf.distribute` 模块中的 `MultiWorkerMirroredStrategy` 将模型训练分布式到多台机器上。

### 数据集

本教程使用的数据集是 GLUE 基准测试中的 [Multi-Genre Natural Language Inference Corpus (MNLI)](https://www.tensorflow.org/datasets/catalog/glue#gluemnli)。从 Cloud Storage 存储桶中加载这个数据集的预处理版本，并用于微调 BERT 模型进行句子预测。

### 目标

在本笔记本中，您将在 Docker 容器中的 Python 脚本中创建一个自定义训练模型。您将学习如何配置、提交和监视一个使用 Reduction Server 优化网络带宽和梯度降低操作的 Vertex Training 作业。

执行的步骤包括：

- 创建一个使用 Reduction Server 的 Vertex 自定义训练作业。
- 提交和监视作业。
- 清理资源。

### 成本

此教程使用 Google Cloud 的付费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

###设置本地开发环境

**如果您正在使用Colab或Google Cloud笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合本笔记本的要求。您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南 [设置Python开发环境](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一套简洁的说明：

1. [安装和初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端shell中运行 `pip3 install jupyter`。

5. 要启动Jupyter，请在终端shell中运行 `jupyter notebook`。

6. 在Jupyter Notebook Dashboard中打开本笔记本。

安装所需的软件包

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

安装Python的最新版本的Vertex SDK。

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新版本的Google Cloud Storage库。

In [ ]:
! pip3 install --upgrade google-cloud-storage $USER_FLAG

### 重新启动内核

安装额外包后，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

设置您的Google Cloud项目

**无论您使用什么笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得$300的免费信用，可用于抵消计算/存储成本。

2. [确保为您的项目启用了计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格以确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter将以`!`前缀的行作为shell命令运行，并将以`$`前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，在这里设置你的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
! gcloud config set project {PROJECT_ID}

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过验证。请跳过此步骤。

如果您正在使用Colab，请运行下面的单元格，并按照提示进行oAuth身份验证。

否则，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐户密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐户**。

3. 在**服务帐户名称**字段中输入名称，然后点击**创建**。

4. 在**授予此服务帐户对项目的访问权限**部分，点击**角色**下拉列表。在筛选框中输入"Vertex AI"，并选择**Vertex AI管理员**。在筛选框中输入"Storage Object Admin"，并选择**存储对象管理员**。

5. 点击*创建*。一个包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中将您的服务帐户密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，并运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建云存储桶

**无论您使用的是哪种笔记本环境，都需要完成以下步骤。**

在此示例中，您的训练应用程序使用云存储来访问训练和验证数据集，并存储检查点。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

您也可以更改`REGION`变量，这将用于笔记本其余部分的操作。确保选择一个[支持 Vertex AI 服务的区域](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。您不可以使用多区域存储桶来训练 Vertex AI。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

只有在您的存储桶尚不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶中的内容验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

导入库并定义常量

In [ ]:
import time

from google.cloud import aiplatform

### 初始化 Vertex SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

### 设置数据集位置

多种类自然语言推理语料库（MNLI）数据集已经被预处理为TensorFlow NLP建模工具包所需的格式，并上传到公共云存储桶中。

In [ ]:
DATASET_LOCATION = "gs://cloud-samples-data/vertex-ai/community-content/datasets/MNLI"
TRAIN_FILE = f"{DATASET_LOCATION}/mnli_train.tf_record"
EVAL_FILE = f"{DATASET_LOCATION}/mnli_valid.tf_record"
METADATA_FILE = f"{DATASET_LOCATION}/metadata.json"

In [ ]:
# List the files

! gsutil ls {DATASET_LOCATION}

In [ ]:
# Examine dataset metadata

! gsutil cat {METADATA_FILE}

### 创建一个训练容器
#### 编写Dockerfile

将代码容器化的第一步是创建一个Dockerfile。在Dockerfile中，您将包含所有需要运行镜像的命令，例如安装必要的库和设置训练代码的入口点。

此Dockerfile使用顶点预构建的TensorFlow 2.5 GPU Docker镜像作为基础镜像。在下载该镜像后，此Dockerfile安装了TensorFlow官方模型和TensorFlow文本库。在基础镜像中预先安装了Reduction Server NCCL插件。

In [ ]:
# Create training image directory

! mkdir training_image

In [ ]:
%%writefile training_image/Dockerfile

FROM us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-5:latest
    
WORKDIR /

# Installs Official Models and Text libraries
RUN pip install tf-models-official==2.5.1 tensorflow-text==2.5.0


# Copies the trainer code to the docker image.
COPY trainer /trainer

#### 创建培训应用程序代码

接下来，您将创建一个名为`training_image/trainer`的目录，其中包含一个名为`train.py`的脚本，该脚本包含了您的培训应用程序的代码。

`train.py`培训脚本基于来自TensorFlow NLP建模工具包的[常用培训驱动程序](https://github.com/tensorflow/models/blob/master/official/nlp/docs/train.md)。常用培训驱动程序脚本支持多个NLP任务（例如，预培训、GLUE和SQuAD微调）和多个模型（例如，BERT、ALBERT）。

针对特定NLP任务的一组配置称为实验。工具包包含一组预定义的实验。当您调用脚本时，您可以使用`--experiment`命令行参数来指定实验类型。有两种覆盖默认实验配置的选项：
- 使用`--config_file`命令行参数指定一个或多个更新设置的YAML配置
- 通过`--params_override`命令行参数将更新的设置提供为键/值对列表

如果同时指定了`--config_file`和`--params_override`，则`--params_override`中的设置优先。

检索默认实验配置并合并用户提供的设置封装在来自TensorFlow Model Garden的`official.core.train_utils.parse_configuration`实用函数中。

在下面的单元格中，[共享培训驱动程序脚本](https://github.com/tensorflow/models/blob/master/official/nlp/train.py)已经经过适应，可以无缝地在运行Vertex培训作业时在分布式计算环境上运行。TensorFlow NLP建模工具包使用[Orbit](https://github.com/tensorflow/models/tree/master/orbit)来实施自定义训练循环。自定义训练循环保存检查点，编写Tensorboard摘要，并将训练好的模型保存到通过`--model-dir`命令行参数指定的存储位置。

请注意，当在分布式设置中使用基本的共享培训驱动程序时，每个工作者都使用相同的基本存储位置。为避免多个工作者同时向同一存储位置写入时发生冲突，驱动程序代码已被修改，使每个工作者根据其在Vertex计算集群中的角色使用不同的存储位置。此逻辑实现在`_get_model_dir`实用函数中。

In [ ]:
# Create trainer directory

! mkdir training_image/trainer

In [ ]:
%%writefile training_image/trainer/train.py

# Copyright 2021 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""TFM common training driver."""

import json
import os

from absl import app
from absl import flags
from absl import logging
import gin
from official.common import distribute_utils
from official.common import flags as tfm_flags
from official.common import registry_imports
from official.core import task_factory
from official.core import train_lib
from official.core import train_utils
from official.modeling import performance

FLAGS = flags.FLAGS


def _get_model_dir(model_dir):
  """Defines utility functions for model saving.

  In a multi-worker scenario, the chief worker will save to the
  desired model directory, while the other workers will save the model to
  temporary directories. It’s important that these temporary directories
  are unique in order to prevent multiple workers from writing to the same
  location. Saving can contain collective ops, so all workers must save and
  not just the chief.
  """

  def _is_chief(task_type, task_id):
    return ((task_type == 'chief' and task_id == 0) or task_type is None)

  tf_config = os.getenv('TF_CONFIG')
  if tf_config:
    tf_config = json.loads(tf_config)

    if not _is_chief(tf_config['task']['type'], tf_config['task']['index']):
      model_dir = os.path.join(model_dir,
                               'worker-{}').format(tf_config['task']['index'])

  logging.info('Setting model_dir to: %s', model_dir)

  return model_dir


def main(_):

  model_dir = _get_model_dir(FLAGS.model_dir)

  gin.parse_config_files_and_bindings(FLAGS.gin_file, FLAGS.gin_params)
  params = train_utils.parse_configuration(FLAGS)

  if 'train' in FLAGS.mode:
    # Pure eval modes do not output yaml files. Otherwise continuous eval job
    # may race against the train job for writing the same file.
    train_utils.serialize_config(params, model_dir)

  # Sets mixed_precision policy. Using 'mixed_float16' or 'mixed_bfloat16'
  # can have significant impact on model speeds by utilizing float16 in case of
  # GPUs, and bfloat16 in the case of TPUs. loss_scale takes effect only when
  # dtype is float16
  if params.runtime.mixed_precision_dtype:
    performance.set_mixed_precision_policy(params.runtime.mixed_precision_dtype)
  distribution_strategy = distribute_utils.get_distribution_strategy(
      distribution_strategy=params.runtime.distribution_strategy,
      all_reduce_alg=params.runtime.all_reduce_alg,
      num_gpus=params.runtime.num_gpus,
      tpu_address=params.runtime.tpu,
      **params.runtime.model_parallelism())
  with distribution_strategy.scope():
    task = task_factory.get_task(params.task, logging_dir=model_dir)

  train_lib.run_experiment(
      distribution_strategy=distribution_strategy,
      task=task,
      mode=FLAGS.mode,
      params=params,
      model_dir=model_dir)

  train_utils.save_gin_config(FLAGS.mode, model_dir)


if __name__ == '__main__':
  tfm_flags.define_flags()
  app.run(main)

####  为MNLI微调实验创建基本设置

TensorFlow NLP建模工具包包括一组文本分类任务的预定义实验，即`bert/sentence_prediction`实验。 `bert/sentence_prediction`实验中的基本设置需要针对您在此示例中执行的MNLI微调任务进行更新。

在下一个单元格中，您可以通过创建一个YAML配置文件来更新这些设置，该文件在调用训练脚本时将被引用。正如前面所述，您可以使用`--params_override`标志进一步微调这些设置，以用于每次训练运行。

配置文件包含三个部分：`task`，`trainer`和`runtime`。
* `task`部分包含特定于机器学习任务的设置，包括指向预训练BERT模型的URI，训练和验证数据集设置以及评估指标。
* `trainer`部分配置控制自定义训练循环的设置，例如检查点间隔或训练步骤数量。
* `runtime`部分包括用于训练运行时的设置：分布式训练策略、GPU配置等。

In [ ]:
%%writefile training_image/trainer/glue_mnli_matched.yaml

task:
  hub_module_url: 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'
  model:
    num_classes: 3
  init_checkpoint: ''
  metric_type: 'accuracy'
  train_data:
    drop_remainder: true
    global_batch_size: 32
    input_path: ''
    is_training: true
    seq_length: 128
    label_type: 'int'
  validation_data:
    drop_remainder: false
    global_batch_size: 32
    input_path: ''
    is_training: false
    seq_length: 128
    label_type: 'int'
trainer:
  checkpoint_interval: 3000
  optimizer_config:
    learning_rate:
      polynomial:
        # 100% of train_steps.
        decay_steps: 36813
        end_learning_rate: 0.0
        initial_learning_rate: 3.0e-05
        power: 1.0
      type: polynomial
    optimizer:
      type: adamw
    warmup:
      polynomial:
        power: 1
        # ~10% of train_steps.
        warmup_steps: 3681
      type: polynomial
  steps_per_loop: 1000
  summary_interval: 1000
  # Training data size 392,702 examples, 3 epochs.
  train_steps: 36813
  validation_interval: 6135
  # Eval data size = 9815 examples.
  validation_steps: 307
  best_checkpoint_export_subdir: 'best_ckpt'
  best_checkpoint_eval_metric: 'cls_accuracy'
  best_checkpoint_metric_comp: 'higher'
runtime:
  distribution_strategy: 'multi_worker_mirrored'
  all_reduce_alg: 'nccl'

### 构建容器

在接下来的单元格中，您将构建容器并推送到Google容器注册表。

In [ ]:
TRAIN_IMAGE = f"gcr.io/{PROJECT_ID}/mnli_finetuning"

In [ ]:
! docker build -t {TRAIN_IMAGE} training_image

In [ ]:
! docker push {TRAIN_IMAGE}

### 创建自定义训练作业

在使用 Vertex AI 运行分布式训练作业时，您需要在一个训练集群中指定多台机器（节点）。训练服务会为您指定的机器类型分配资源。在特定节点上运行的作业被称为一个副本。具有相同配置的一组副本被称为一个 worker 池。Vertex Training 提供了 4 个 worker 池，用于执行不同类型的机器任务。要使用 Reduction Server，您需要使用其中的 3 个 worker 池中的一个。

* **Worker 池 0** 配置主要节点、主导节点、调度节点或“主节点”。这个 worker 通常会执行一些额外的工作，如保存检查点和编写摘要文件。在一个集群中只会有一个主节点，因此您在 worker 池 0 中的 worker 计数始终为 1。

* **Worker 池 1** 是配置集群中其余工作者的地方。

* **Worker 池 2** 管理 Reduction Server 的 reducer。

Worker 池 0 和 1 运行您在上一步创建的自定义训练容器。Worker 池 2 使用 Vertex AI 提供的 Reduction Server 镜像。

下面的辅助函数使用描述的 worker 池拓扑来创建 worker 池规格。

In [ ]:
def prepare_worker_pool_specs(
    image_uri,
    args,
    cmd,
    replica_count=1,
    machine_type="n1-standard-4",
    accelerator_count=0,
    accelerator_type="ACCELERATOR_TYPE_UNSPECIFIED",
    reduction_server_count=0,
    reduction_server_machine_type="n1-highcpu-16",
    reduction_server_image_uri="us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest",
):

    if accelerator_count > 0:
        machine_spec = {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        }
    else:
        machine_spec = {"machine_type": machine_type}

    container_spec = {
        "image_uri": image_uri,
        "args": args,
        "command": cmd,
    }

    chief_spec = {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "container_spec": container_spec,
    }

    worker_pool_specs = [chief_spec]
    if replica_count > 1:
        workers_spec = {
            "replica_count": replica_count - 1,
            "machine_spec": machine_spec,
            "container_spec": container_spec,
        }
        worker_pool_specs.append(workers_spec)

    if reduction_server_count > 1:
        workers_spec = {
            "replica_count": reduction_server_count,
            "machine_spec": {
                "machine_type": reduction_server_machine_type,
            },
            "container_spec": {"image_uri": reduction_server_image_uri},
        }
        worker_pool_specs.append(workers_spec)

    return worker_pool_specs

配置工作池

在选择减少器的数量和类型时，您应考虑减少器副本的机器类型支持的网络带宽。在GCP中，虚拟机的机器类型定义了其最大可能的出口带宽。例如，`n1-highcpu-16` 机器类型的出口带宽被限制在 32 Gbps。

由于减少器执行非常有限的功能，即聚合梯度块，因此它们可以在相对低功耗和成本效益的机器上运行。即使有大量梯度，此计算也不需要加速硬件或高CPU或内存资源。但是，为避免网络瓶颈，减少器工作池中所有副本的总聚合带宽必须大于或等于主机 GPU 工作器池 0 和 1 中所有副本的总聚合带宽。

In [ ]:
REPLICA_COUNT = 2
WORKER_MACHINE_TYPE = "a2-highgpu-4g"
ACCELERATOR_TYPE = "NVIDIA_TESLA_A100"
PER_MACHINE_ACCELERATOR_COUNT = 4
PER_REPLICA_BATCH_SIZE = 32

REDUCTION_SERVER_COUNT = 4
REDUCTION_SERVER_MACHINE_TYPE = "n1-highcpu-16"

调整 MNLI 实验设置

MNLI 微调实验的默认设置已经在之前步骤中创建的 YAML 配置文件中进行了配置。要覆盖特定训练运行的默认设置，请使用 `--params_override` 标志。

`params_override` 接受一个包含每个要覆盖的参数的逗号分隔键/值对的字符串。

在下一个单元格中，您可以更新以下设置：

- `trainer.train_step` - 训练步数。
- `trainer.steps_per_loop` - 训练脚本每 `steps_per_loop` 打印出关于训练进度的更新。
- `trainer.summary_interval` - 训练脚本每 `summary_interval` 记录 Tensorboard 摘要。
- `trainer.validation_interval` - 训练脚本每 `validation_interval` 运行一次验证。
- `trainer.checkpoint_interval` - 训练脚本每 `checkpoint_interval` 创建一个检查点。
- `task.train_data.global_batch_size` - 训练数据的全局批量大小。
- `task.validation_data.global_batch_size` - 验证数据的全局批量大小。
- `task.train_data.input_path` - 训练数据集的位置。
- `task.validation_data.input_path` - 验证数据集的位置。
- `runtime.num_gpus` - 每个工作节点要使用的 GPU 数量。

In [ ]:
PARAMS_OVERRIDE = ",".join(
    [
        "trainer.train_steps=2000",
        "trainer.steps_per_loop=100",
        "trainer.summary_interval=100",
        "trainer.validation_interval=2000",
        "trainer.checkpoint_interval=2000",
        "task.train_data.global_batch_size="
        + str(REPLICA_COUNT * PER_REPLICA_BATCH_SIZE * PER_MACHINE_ACCELERATOR_COUNT),
        "task.validation_data.global_batch_size="
        + str(REPLICA_COUNT * PER_REPLICA_BATCH_SIZE * PER_MACHINE_ACCELERATOR_COUNT),
        "task.train_data.input_path=" + TRAIN_FILE,
        "task.validation_data.input_path=" + EVAL_FILE,
        "runtime.num_gpus=" + str(PER_MACHINE_ACCELERATOR_COUNT),
    ]
)

### 提交和监控作业

在实验和工作人员池配置参数已经定义好之后，使用 Python 的 Vertex SDK 来提交和监控训练作业。

In [ ]:
JOB_NAME = "MNLI_{}".format(time.strftime("%Y%m%d_%H%M%S"))
MODEL_DIR = f"{BUCKET_NAME}/{JOB_NAME}/model"

WORKER_CMD = ["python", "trainer/train.py"]
WORKER_ARGS = [
    "--experiment=bert/sentence_prediction",
    "--mode=train",
    "--model_dir=" + MODEL_DIR,
    "--config_file=trainer/glue_mnli_matched.yaml",
    "--params_override=" + PARAMS_OVERRIDE,
]

worker_pool_specs = prepare_worker_pool_specs(
    image_uri=TRAIN_IMAGE,
    args=WORKER_ARGS,
    cmd=WORKER_CMD,
    replica_count=REPLICA_COUNT,
    machine_type=WORKER_MACHINE_TYPE,
    accelerator_count=PER_MACHINE_ACCELERATOR_COUNT,
    accelerator_type=ACCELERATOR_TYPE,
    reduction_server_count=REDUCTION_SERVER_COUNT,
    reduction_server_machine_type=REDUCTION_SERVER_MACHINE_TYPE,
)

In [ ]:
job = aiplatform.CustomJob(
    display_name=JOB_NAME,
    worker_pool_specs=worker_pool_specs,
)
job.run(sync=False)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

In [ ]:
# Delete Cloud Storage objects that were created
! gsutil -m rm -r {BUCKET_NAME}

# Delete the training job
delete_training_job = True
job.delete()